In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [2]:
from secret_keys import openai_apikey1

os.environ['OPENAI_API_KEY']= openai_apikey1

In [3]:
llm=OpenAI(temperature=0.9, max_tokens=500)

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

C:\Users\priya\AppData\Local\Temp\ipykernel_19388\2900312403.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(temperature=0.9, max_tokens=500)


2

In [4]:
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs=text_splitter.split_documents(data)
len(docs)

15

In [5]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹15 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_MARKETS_AS/MC_ENG_ROS_NWS_MKTS_AS_ATF_728\n\nGo PRO @₹99 PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessMarketsWall Street rises as Tesla soars on AI optimism\n\nTrending Topics\n\nSensex TodayAmber Enterprises share priceMT

In [6]:
embeddings =OpenAIEmbeddings()

vectorindex_openai=FAISS.from_documents(docs, embeddings)

C:\Users\priya\AppData\Local\Temp\ipykernel_19388\4186546262.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings =OpenAIEmbeddings()


In [7]:
texts = [doc.page_content for doc in docs]

In [8]:
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(texts, f)

In [9]:
vectorindex_openai.save_local("faiss_index")

In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex=pickle.load(f)

In [11]:
docs = [Document(page_content=text) for text in texts]

In [12]:
vectorindex_openai = FAISS.load_local("faiss_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [13]:
retriever = vectorindex_openai.as_retriever()

In [14]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)

In [15]:
query="what is the price of Tiago iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

C:\Users\priya\AppData\Local\Temp\ipykernel_19388\3411104888.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"question": query}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/error] [chain:RetrievalQAWithSourcesChain] s] Chain run errored with error:
"AuthenticationError(\"Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************wZ0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}\")Traceback (most recent call last):\n\n\n  File \"C:\\Users\\priya\\anaconda3\\Lib\\site-packages\\langchain\\chains\\base.py\", line 160, in invoke\n    self._call(inputs, run_manager=run_manager)\n\n\n  File \"C:\\Users\\priya\\anaconda3\\Lib\\site-packages\\langchain\\chains\\qa_with_sources\\base.py\", line 162, in _call\n    docs = self._get_docs(inputs, run_manag

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************wZ0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}